In [1]:
%load_ext autoreload
%load_ext autotime

time: 404 µs (started: 2023-03-12 13:46:45 -07:00)


In [2]:
%run init.ipynb

time: 2.53 s (started: 2023-03-12 13:46:45 -07:00)


In [3]:
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

time: 15.6 ms (started: 2023-03-12 13:46:47 -07:00)


In [4]:
import dask
import dask.array as da
import dask.bag as db
import dask.dataframe as dd
import json
import math
import numpy as np
import os
import pandas as pd
# import pyarrow as pa
from dask import compute, delayed
from dask.dataframe import DataFrame
from dask.distributed import Client, LocalCluster, fire_and_forget, wait, worker_client
from wisio.utils.file_utils import ensure_dir
from wisio.utils.json_encoders import NpEncoder
from wisio.utils.logger import create_logger, format_log

time: 652 ms (started: 2023-03-12 13:46:47 -07:00)


In [5]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.LSFCluster
Dashboard: http://192.168.65.157:8788/status,
Dashboard: http://192.168.65.157:8788/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://192.168.65.157:43575,Workers: 0
Dashboard: http://192.168.65.157:8788/status,Total threads: 0
Started: Just now,Total memory: 0 B


time: 69.5 ms (started: 2023-03-12 13:46:48 -07:00)


In [6]:
import json

def load_global_min_max(log_dir: str):
    with open(f"{log_dir}/global.json") as file:
        global_min_max = json.load(file)
    return global_min_max

time: 944 µs (started: 2023-03-12 13:46:48 -07:00)


In [7]:
log_dir = "/p/gpfs1/iopp/recorder_app_logs/lbann-jag/nodes-32/_parquet"
ddf = dd.read_parquet(f"{log_dir}/*.parquet")
ddf 

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,proc_name,file_name,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id
npartitions=3840,,,,,,,,,,,,,,,,,,,,,
,int64,int64,int32,int32,int32,int32,float32,float32,object,int32,object,object,object,object,int64,int32,float32,float32,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


time: 487 ms (started: 2023-03-12 13:46:48 -07:00)


In [8]:
ddf.head()

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,...,app,proc_name,file_name,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id
0,1,1,0,-2127943504,2,0,0.178566,2.712372,MPI_Comm_dup,0,...,app1,,UNKNOWN,0,0,0.0,2.533806e+00,14454688,0,0
1,2,1,0,-2127943504,2,0,2.712592,2.712593,MPI_Comm_rank,0,...,app1,,UNKNOWN,0,0,0.0,9.000000e-07,27125922,0,0
2,3,1,0,-2127943504,2,0,2.712597,2.712597,MPI_Comm_size,0,...,app1,,UNKNOWN,0,0,0.0,0.000000e+00,27125968,0,0
3,4,1,0,-2127943504,2,0,2.712606,2.713452,MPI_Comm_split_type,0,...,app1,,UNKNOWN,0,0,0.0,8.459000e-04,27130287,0,0
4,5,1,0,-2127943504,2,0,2.713541,2.713541,MPI_Comm_rank,0,...,app1,,UNKNOWN,0,0,0.0,0.000000e+00,27135408,0,0


time: 49.4 s (started: 2023-03-12 13:43:18 -07:00)


In [ ]:
ddf.index.count().compute()

In [ ]:
ddf['tend'].max().compute()

In [ ]:
ddf['file_id'].unique().count().compute()

In [ ]:
ddf['proc_id'].unique().count().compute()

In [8]:
global_min_max = load_global_min_max(log_dir=log_dir)
global_min_max

{'file_id': [113104469769545699, 7751659605141216184],
 'proc_id': [-929352282003194517, -929352281994825296],
 'tmid': [0, 1943551213547325]}

time: 4.78 ms (started: 2023-03-12 13:46:53 -07:00)


In [9]:
from dask.distributed import Future, get_client
from typing import Union

DELTA_BINS = [
    0,
    0.001,
    0.01,
    0.1,
    0.25,
    0.5,
    0.75,
    1
]
DELTA_BIN_LABELS = [
    'none',
    'trivial',
    'very low',
    'low',
    'medium',
    'high',
    'very high',
    'critical'
]
TIME_PRECISION = 1e7

def set_metric_deltas(df: pd.DataFrame, metric: str, max_io_time: float):
    metric_col, csp_col, delta_col = (
        f"{metric}_sum",
        f"{metric}_csp",
        f"{metric}_delta",
    )
    df[csp_col] = df[metric_col].cumsum() / max_io_time
    df[delta_col] = df[csp_col].diff().fillna(df[csp_col])
    return df


def set_metric_percentages(df: pd.DataFrame, metric: str, max_io_time: float):
    metric_col, pero_col, perr_col = (
        f"{metric}_sum",
        f"{metric}_pero",
        f"{metric}_perr"
    )
    df[pero_col] = df[metric_col] / max_io_time
    df[perr_col] = df[metric_col] / df[metric_col].sum()
    return df


def set_metric_scores(df: pd.DataFrame, metric: str, col: str):
    bin_col, score_col, threshold_col = (
        f"{metric}_bin",
        f"{metric}_score",
        f"{metric}_th",
    )
    df[bin_col] = np.digitize(df[col], bins=DELTA_BINS, right=True)
    df[score_col] = np.choose(df[bin_col] - 1, choices=DELTA_BIN_LABELS, mode='clip')
    df[threshold_col] = np.choose(df[bin_col] - 1, choices=DELTA_BINS, mode='clip')
    return df.drop(columns=[bin_col])

def compute_tranges(global_min_max: dict, precision=TIME_PRECISION):
    tmid_min, tmid_max = global_min_max['tmid']
    tranges = np.arange(tmid_min, tmid_max, precision)
    return get_client().scatter(tranges)


def set_tranges(df: pd.DataFrame, tranges: Union[Future, np.ndarray]):
    tranges = tranges.result() if isinstance(tranges, Future) else tranges
    return df.assign(trange=np.digitize(df['tmid'], bins=tranges, right=True))

time: 4.34 ms (started: 2023-03-12 13:46:54 -07:00)


In [10]:
tranges = compute_tranges(global_min_max=global_min_max)
tranges

<Future: finished, type: numpy.ndarray, key: ndarray-bb64ea7f148802b5f2d3df14260739f9>

time: 20.6 s (started: 2023-03-12 13:46:55 -07:00)


In [11]:
VIEW_TYPES = ['trange', 'file_name', 'proc_name']

time: 665 µs (started: 2023-03-12 13:47:16 -07:00)


In [13]:
from typing import Iterable

def deepflatten(collection, ignore_types=(bytes, str)):
    for x in collection:
        if isinstance(x, Iterable) and not isinstance(x, ignore_types):
            yield from deepflatten(x)
        else:
            yield x

def unique_flatten():
    return dd.Aggregation(
        'unique_flatten',
        lambda s: s.apply(lambda x: np.unique(x).tolist()),
        lambda s0: s0.apply(lambda x: np.unique(x).tolist()),
        lambda s1: s1.apply(lambda x: np.unique(list(deepflatten(x))).tolist()),
    )

time: 2.35 ms (started: 2023-03-12 13:44:28 -07:00)


In [12]:
from copy import copy

view_types = copy(VIEW_TYPES)
view_types.append('io_cat')
view_types.append('acc_pat')
view_types.append('func_id')

main_view = ddf[(ddf['cat'] == 0) & (ddf['io_cat'].isin([1,2,3]))]  \
    .map_partitions(set_tranges, tranges=tranges) \
    .groupby(view_types) \
    .agg({
        'duration': [sum],
        'index': ['count'],
        'size': [min, max, sum],
#         'file_name': ['first'],
#         'proc_name': ['first'],
#         'func_id': [unique_flatten()],
    }) \
    .persist()

main_view


Dask DataFrame Structure:
              duration  index   size              
                   sum  count    min    max    sum
npartitions=1                                     
               float32  int64  int64  int64  int64
                   ...    ...    ...    ...    ...
Dask Name: aggregate-agg, 1 tasks

time: 15.5 s (started: 2023-03-12 13:47:16 -07:00)


In [13]:
wait(main_view)

DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('aggregate-agg-0967eb0a49fc703e7374ce1624d836d7', 0)>}, not_done=set())

time: 5min 5s (started: 2023-03-12 13:47:31 -07:00)


In [14]:
main_view = main_view.reset_index()
main_view.columns = ['_'.join(tup).rstrip('_') for tup in main_view.columns.values]
main_view = main_view.persist()

time: 40.9 ms (started: 2023-03-12 13:52:52 -07:00)


In [15]:
wait(main_view)

DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('rename-c1d5222d09451e99a12e293e8a824fe8', 0)>}, not_done=set())

time: 10.2 ms (started: 2023-03-12 13:52:53 -07:00)


In [18]:
!rm -rf {log_dir}/cached & mkdir -p {log_dir}/cached

time: 141 ms (started: 2023-03-12 13:54:03 -07:00)


In [19]:
main_view.repartition('100MB').to_parquet(f"{log_dir}/cached/main_view")

[None]

time: 245 ms (started: 2023-03-12 13:54:05 -07:00)


In [22]:
!ls -lh {log_dir}/cached/main_view

total 1.6M
-rw------- 1 izzet iopp 2.0K Mar 12 13:54 _common_metadata
-rw------- 1 izzet iopp 3.2K Mar 12 13:54 _metadata
-rw------- 1 izzet iopp 1.6M Mar 12 13:54 part.0.parquet
time: 142 ms (started: 2023-03-12 13:55:18 -07:00)


In [20]:
main_view_cached = dd.read_parquet(f"{log_dir}/cached/main_view/*.parquet")
# main_view_cached['func_id_unique_flatten'] = main_view_cached['func_id_unique_flatten'].astype(list)
main_view_cached.head()

,trange,file_name,proc_name,io_cat,acc_pat,func_id,duration_sum,index_count,size_min,size_max,size_sum
0,3,UNKNOWN,app1#localhost#0#35187479025840,3,0,closedir,0.000026,9,0,0,0
1,3,UNKNOWN,app1#localhost#0#35187479025840,3,0,readdir,0.000391,221,0,0,0
2,3,UNKNOWN,app1#localhost#1#35187478763696,3,0,closedir,0.000026,9,0,0,0
3,3,UNKNOWN,app1#localhost#1#35187478763696,3,0,readdir,0.000320,221,0,0,0
4,3,UNKNOWN,app1#localhost#10#35187478763696,3,0,closedir,0.000025,9,0,0,0


time: 217 ms (started: 2023-03-12 13:54:11 -07:00)


In [25]:
main_view['read_count'] = 0
main_view['write_count'] = 0
main_view['metadata_count'] = 0
main_view['read_count'] = main_view['read_count'].mask(main_view['io_cat'] == 1, main_view['index_count'])
main_view['write_count'] = main_view['write_count'].mask(main_view['io_cat'] == 2,  main_view['index_count'])
main_view['metadata_count'] = main_view['metadata_count'].mask(main_view['io_cat'] == 3,  main_view['index_count'])

main_view['read_size'] = 0
main_view['write_size'] = 0
main_view['metadata_size'] = 0
main_view['read_size'] = main_view['read_size'].mask(main_view['io_cat'] == 1, main_view['size_sum'])
main_view['write_size'] = main_view['write_size'].mask(main_view['io_cat'] == 2,  main_view['size_sum'])
main_view['metadata_size'] = main_view['metadata_size'].mask(main_view['io_cat'] == 3,  main_view['size_sum'])

main_view['read_time'] = 0
main_view['write_time'] = 0
main_view['metadata_time'] = 0
main_view['read_time'] = main_view['read_time'].mask(main_view['io_cat'] == 1, main_view['duration_sum'])
main_view['write_time'] = main_view['write_time'].mask(main_view['io_cat'] == 2,  main_view['duration_sum'])
main_view['metadata_time'] = main_view['metadata_time'].mask(main_view['io_cat'] == 3,  main_view['duration_sum'])


main_view['sequential_count'] = 0
main_view['random_count'] = 0
main_view['sequential_count'] = main_view['sequential_count'].mask(main_view['acc_pat'] == 0, main_view['index_count'])
main_view['random_count'] = main_view['random_count'].mask(main_view['acc_pat'] == 1, main_view['index_count'])

main_view['sequential_size'] = 0
main_view['random_size'] = 0
main_view['sequential_size'] = main_view['sequential_size'].mask(main_view['acc_pat'] == 0, main_view['size_sum'])
main_view['random_size'] = main_view['random_size'].mask(main_view['acc_pat'] == 1, main_view['size_sum'])

main_view['sequential_time'] = 0
main_view['random_time'] = 0
main_view['sequential_time'] = main_view['sequential_time'].mask(main_view['acc_pat'] == 0, main_view['duration_sum'])
main_view['random_time'] = main_view['random_time'].mask(main_view['acc_pat'] == 1, main_view['duration_sum'])


main_view['data_count'] = main_view['write_count'] + main_view['read_count']
main_view['data_size'] = main_view['write_size'] + main_view['read_size']
main_view['data_time'] = main_view['write_time'] + main_view['read_time']

main_view['open_time'] = 0
main_view['close_time'] = 0
main_view['seek_time'] = 0
main_view['stat_time'] = 0
main_view['open_time'] = main_view['open_time'].mask(main_view['func_id'].str.contains('open') & ~main_view['func_id'].str.contains('dir'),  main_view['duration_sum'])
main_view['close_time'] = main_view['close_time'].mask(main_view['func_id'].str.contains('close') & ~main_view['func_id'].str.contains('dir'),  main_view['duration_sum'])
main_view['seek_time'] = main_view['seek_time'].mask(main_view['func_id'].str.contains('seek'),  main_view['duration_sum'])
main_view['stat_time'] = main_view['stat_time'].mask(main_view['func_id'].str.contains('stat'),  main_view['duration_sum'])

final_view = main_view \
    .groupby(VIEW_TYPES) \
    .sum() \
    .persist()


time: 771 ms (started: 2023-03-12 13:55:52 -07:00)


In [26]:
wait(final_view)

DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('dataframe-groupby-sum-agg-9f3a29703f1096bd4d29cd2eba1ed4e5', 0)>}, not_done=set())

time: 14.2 ms (started: 2023-03-12 13:55:53 -07:00)


In [27]:
final_view = final_view \
    .reset_index() \
    .rename(columns={
        'file_name_first': 'file_name', 
        'proc_name_first': 'proc_name',
    }) \
    .persist()

time: 43.9 ms (started: 2023-03-12 13:55:58 -07:00)


In [28]:
wait(final_view)

DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('rename-2decbf70e16d78efd158846b6e095642', 0)>}, not_done=set())

time: 4.35 ms (started: 2023-03-12 13:55:59 -07:00)


In [29]:
final_view.head()

,trange,file_name,proc_name,io_cat,acc_pat,duration_sum,index_count,size_min,size_max,size_sum,...,random_size,sequential_time,random_time,data_count,data_size,data_time,open_time,close_time,seek_time,stat_time
0,3,UNKNOWN,app1#localhost#0#35187479025840,6,0,0.000417,230,0,0,0,...,0,0.000417,0,0,0,0.0,0.0,0.0,0,0
1,3,UNKNOWN,app1#localhost#1#35187478763696,6,0,0.000346,230,0,0,0,...,0,0.000346,0,0,0,0.0,0.0,0.0,0,0
2,3,UNKNOWN,app1#localhost#10#35187478763696,6,0,0.000338,230,0,0,0,...,0,0.000338,0,0,0,0.0,0.0,0.0,0,0
3,3,UNKNOWN,app1#localhost#100#35187478763696,6,0,0.000527,230,0,0,0,...,0,0.000527,0,0,0,0.0,0.0,0.0,0,0
4,3,UNKNOWN,app1#localhost#101#35187478763696,6,0,0.000528,230,0,0,0,...,0,0.000528,0,0,0,0.0,0.0,0.0,0,0


time: 62.9 ms (started: 2023-03-12 13:56:00 -07:00)


In [23]:
del main_view

time: 777 µs (started: 2023-03-12 11:34:33 -07:00)


In [24]:
final_view.head()

,trange,file_id,proc_id,duration_sum,index_count,size_sum,read_count,write_count,metadata_count,read_size,...,random_size,sequential_time,random_time,data_count,data_size,data_time,size_min,size_max,file_name,proc_name
0,3,113104469769545699,-929352282003156512,0.000417,230,0,0,0,230,0,...,0,0.000417,0,0,0,0.0,0,0,UNKNOWN,app1#localhost#0#35187479025840
1,3,113104469769545699,-929352282003090980,0.000346,230,0,0,0,230,0,...,0,0.000346,0,0,0,0.0,0,0,UNKNOWN,app1#localhost#1#35187478763696
2,3,113104469769545699,-929352282003025444,0.000383,230,0,0,0,230,0,...,0,0.000383,0,0,0,0.0,0,0,UNKNOWN,app1#localhost#2#35187478763696
3,3,113104469769545699,-929352282002959908,0.000383,230,0,0,0,230,0,...,0,0.000383,0,0,0,0.0,0,0,UNKNOWN,app1#localhost#3#35187478763696
4,3,113104469769545699,-929352282002894372,0.000456,230,0,0,0,230,0,...,0,0.000456,0,0,0,0.0,0,0,UNKNOWN,app1#localhost#4#35187478763696


time: 35.8 ms (started: 2023-03-12 11:34:33 -07:00)


In [30]:
max_io_time = final_view.groupby(['proc_name']).agg({'duration_sum':sum})['duration_sum'].max()
max_io_time

dd.Scalar<series-..., dtype=float32>

time: 29.1 ms (started: 2023-03-12 13:56:13 -07:00)


In [31]:
DELTA = 0.0001
CUT = 0.5

time: 750 µs (started: 2023-03-12 13:56:14 -07:00)


In [32]:
import itertools as it

def _view_permutations(r: int):
    return it.permutations(VIEW_TYPES, r + 1)

metric = 'duration'
views = {}
for view_permutation in it.chain.from_iterable(map(_view_permutations, range(len(VIEW_TYPES)))):
    # Read types
    parent_type = view_permutation[:-1]
    view_type = view_permutation[-1]
    # Get parent view
    parent_view = views[parent_type] if parent_type in views else final_view
    # Create colum names
    metric_col, delta_col = f"{metric}_sum", f"{metric}_delta"
    # Check view type
    if view_type is not 'proc_name':
        # Compute `proc_id` view first
        group_view = parent_view \
            .groupby([view_type, 'proc_name']) \
            .agg({metric_col: sum}) \
            .groupby([view_type]) \
            .max()
    else:
        # Compute group view
        group_view = parent_view \
            .groupby([view_type]) \
            .agg({metric_col: sum})
    # Set metric scores
    group_view = group_view \
        .map_partitions(set_metric_deltas, metric=metric, max_io_time=max_io_time) \
        .query(f"{delta_col} > @delta", local_dict={'delta': DELTA})
    # Find filtered records
    view = parent_view.query(f"{view_type} in @indices", local_dict={'indices': group_view.index.unique()})
    # Return views
    views[view_permutation] = view

len(views)

15

time: 856 ms (started: 2023-03-12 13:56:23 -07:00)


In [33]:
for view_key, view in views.items():
    print(view_key, len(view))

('trange',) 5982
('file_name',) 6377
('proc_name',) 1180
('trange', 'file_name') 657
('trange', 'proc_name') 785
('file_name', 'trange') 5982
('file_name', 'proc_name') 1052
('proc_name', 'trange') 785
('proc_name', 'file_name') 1052
('trange', 'file_name', 'proc_name') 657
('trange', 'proc_name', 'file_name') 657
('file_name', 'trange', 'proc_name') 785
('file_name', 'proc_name', 'trange') 785
('proc_name', 'trange', 'file_name') 657
('proc_name', 'file_name', 'trange') 785
time: 2.9 s (started: 2023-03-12 13:56:26 -07:00)


In [36]:
import dask.dataframe as dd
import pandas as pd
from dask import compute, delayed
from dask.distributed import get_client
from typing import Any, Dict

BOTTLENECK_ORDER = dict(
    file_name=('file_name', 'proc_name', 'trange'),
    proc_name=('proc_name', 'trange', 'file_name'),
    trange=('trange', 'proc_name', 'file_name'),
)
BOTTLENECK_TYPE = dict(
    file_name='file_name',
    proc_name='proc_name',
    trange='trange'
)

def _calculate_llc(level_row: pd.Series):
    llc = dict(level_row)
    for view_type in VIEW_TYPES:
        if view_type in llc:
            llc.pop(view_type)
        bottleneck_type = BOTTLENECK_TYPE[view_type]
        if bottleneck_type in llc:
            llc.pop(bottleneck_type)
    return llc


@delayed
def _process_bottleneck_view(
    view_key: tuple,
    threshold: float,
    low_level_view: pd.DataFrame,
    mid_level_view: pd.DataFrame,
    high_level_view: pd.DataFrame,
):
    # Get view type
    view_type = view_key[-1]
    # Get ordered bottleneck columns
    _, ml_col, ll_col = BOTTLENECK_ORDER[view_type]
    # Init bottlenecks
    bottlenecks = {}
    # Loop through index tuples
    ids_tuple = low_level_view.index
    for hl_id, ml_id, ll_id in ids_tuple:
        hl_row = high_level_view.loc[hl_id]
        ml_row = mid_level_view.loc[(hl_id, ml_id)]
        ll_row = low_level_view.loc[(hl_id, ml_id, ll_id)]
        if hl_id not in bottlenecks:
            bottlenecks[hl_id] = {}
            bottlenecks[hl_id]['llc'] = _calculate_llc(hl_row)
            bottlenecks[hl_id][ml_col] = {}
        if ml_id not in bottlenecks[hl_id][ml_col]:
            bottlenecks[hl_id][ml_col][ml_id] = {}
            bottlenecks[hl_id][ml_col][ml_id]['llc'] = _calculate_llc(ml_row)
            bottlenecks[hl_id][ml_col][ml_id][ll_col] = {}
        if ll_id not in bottlenecks[hl_id][ml_col][ml_id][ll_col]:
            bottlenecks[hl_id][ml_col][ml_id][ll_col][ll_id] = {}
            bottlenecks[hl_id][ml_col][ml_id][ll_col][ll_id]['llc'] = _calculate_llc(ll_row)
    # Return view key & bottlenecks
    return view_key, threshold, bottlenecks

def detect_bottlenecks(views: Dict[tuple, dd.DataFrame], view_types: list) -> Dict[tuple, object]:
    # Keep bottleneck views
    bottleneck_views = {}
    # Run through views
    for view_key, view in views.items():
        # Generate bottleneck views
        bottleneck_views[view_key] = _generate_bottlenecks_views(
            view_key=view_key,
            view=view,
            view_types=view_types,
            max_io_time=max_io_time,
            metric=metric,
        )
    # Generate bottlenecks
    bottlenecks = _process_bottleneck_views(
        bottleneck_views=bottleneck_views,
        metric=metric,
    )
    # Return bottleneck views
    return bottlenecks, bottleneck_views

def _process_bottleneck_views(bottleneck_views: Dict[tuple, dd.DataFrame], metric: str):
    # Init bottlenecks
    bottlenecks = {}
    bottlenecks_delayed = []
    # Run through bottleneck views
    for view_key, view_dict in bottleneck_views.items():
        # For given thresholds
        for th in DELTA_BINS[3:-1]:  # [0.25, 0.5, 0.75]

            threshold_col = f"{metric}_th"

            low_level_view = view_dict['low_level_view']
            mid_level_view = view_dict['mid_level_view']
            high_level_view = view_dict['high_level_view']

            bottlenecks_delayed.append(_process_bottleneck_view(
                view_key=view_key,
                threshold=th,
                low_level_view=low_level_view.query(f"{threshold_col} >= @th", local_dict={'th': th}),
                mid_level_view=mid_level_view.query(f"{threshold_col} >= @th", local_dict={'th': th}),
                high_level_view=high_level_view.query(f"{threshold_col} >= @th", local_dict={'th': th})
            ))

    # Compute all bottlenecks
    results = compute(*bottlenecks_delayed)
    for view_key, th, result in results:
        bottlenecks[view_key] = bottlenecks[view_key] if view_key in bottlenecks else {}
        bottlenecks[view_key][f"{th:.2f}"] = result
    # Return all bottlenecks
    return bottlenecks

def _generate_bottlenecks_views(
    view_key: tuple,
    view: dd.DataFrame,
    view_types: list,
    max_io_time: dd.core.Scalar,
    metric: str,
):
    # Get view type
    view_type = view_key[-1]
    bottleneck_type = BOTTLENECK_TYPE[view_type]

    # Create lower level view
    low_level_view = view \
        .groupby(list(BOTTLENECK_ORDER[view_type])) \
        .first() \
        .drop(columns=['acc_pat', 'io_cat', 'file_id', 'proc_id'], errors='ignore')

    # Non-proc agg columns
    non_proc_agg_dict = _get_agg_dict(view_types=view_types, view_columns=low_level_view.columns, is_proc=False)
    proc_agg_dict = _get_agg_dict(view_types=view_types, view_columns=low_level_view.columns, is_proc=True)

    # Create mid and high level views
    if bottleneck_type is not 'proc_name':
        mid_level_view = low_level_view \
            .reset_index() \
            .groupby([bottleneck_type, 'proc_name']) \
            .agg(non_proc_agg_dict)

        high_level_view = mid_level_view \
            .reset_index() \
            .groupby([bottleneck_type]) \
            .agg(proc_agg_dict)
    else:
        mid_level_view = low_level_view \
            .reset_index() \
            .groupby([bottleneck_type, 'trange']) \
            .agg(non_proc_agg_dict)

        high_level_view = mid_level_view \
            .reset_index() \
            .groupby([bottleneck_type]) \
            .agg(non_proc_agg_dict)

    low_level_view = low_level_view \
        .map_partitions(set_metric_percentages, metric=metric, max_io_time=max_io_time) \
        .map_partitions(set_metric_scores, metric=metric, col=f"{metric}_pero")

    mid_level_view = mid_level_view \
        .map_partitions(set_metric_percentages, metric=metric, max_io_time=max_io_time) \
        .map_partitions(set_metric_scores, metric=metric, col=f"{metric}_pero")

    high_level_view = high_level_view \
        .map_partitions(set_metric_percentages, metric=metric, max_io_time=max_io_time) \
        .map_partitions(set_metric_scores, metric=metric, col=f"{metric}_pero")

    return dict(
        low_level_view=low_level_view,
        mid_level_view=mid_level_view,
        high_level_view=high_level_view
    )

def _get_agg_dict(view_types: list, view_columns: list, is_proc=False):
    if is_proc:
        agg_dict = {col: max if any(x in col for x in 'duration time'.split()) else sum for col in view_columns}
    else:
        agg_dict = {col: sum for col in view_columns}
#     agg_dict['func_id'] = unique_flatten()
    agg_dict['size_min'] = min
    agg_dict['size_max'] = max
    for view_type in view_types:
        if view_type in agg_dict:
            agg_dict.pop(view_type)
        bottleneck_type = BOTTLENECK_TYPE[view_type]
        if bottleneck_type in agg_dict:
            agg_dict.pop(bottleneck_type)
    return agg_dict


time: 11.8 ms (started: 2023-03-12 13:56:47 -07:00)


In [37]:
bottlenecks, bottleneck_views = detect_bottlenecks(
    views=views,
    view_types=VIEW_TYPES,
)

len(bottlenecks)

15

time: 11.9 s (started: 2023-03-12 13:56:48 -07:00)


In [49]:
max_io_time.compute()

8.065207481384277

time: 36.8 ms (started: 2023-03-12 12:00:38 -07:00)


In [38]:
for view_key, bottleneck in bottlenecks.items():
    for th in bottleneck.keys():
        print(view_key, th, len(bottleneck[th]))

('trange',) 0.10 4
('trange',) 0.25 4
('trange',) 0.50 2
('trange',) 0.75 2
('file_name',) 0.10 1
('file_name',) 0.25 1
('file_name',) 0.50 1
('file_name',) 0.75 1
('proc_name',) 0.10 128
('proc_name',) 0.25 127
('proc_name',) 0.50 46
('proc_name',) 0.75 45
('trange', 'file_name') 0.10 1
('trange', 'file_name') 0.25 1
('trange', 'file_name') 0.50 1
('trange', 'file_name') 0.75 1
('trange', 'proc_name') 0.10 128
('trange', 'proc_name') 0.25 127
('trange', 'proc_name') 0.50 46
('trange', 'proc_name') 0.75 45
('file_name', 'trange') 0.10 4
('file_name', 'trange') 0.25 4
('file_name', 'trange') 0.50 2
('file_name', 'trange') 0.75 2
('file_name', 'proc_name') 0.10 128
('file_name', 'proc_name') 0.25 127
('file_name', 'proc_name') 0.50 46
('file_name', 'proc_name') 0.75 45
('proc_name', 'trange') 0.10 4
('proc_name', 'trange') 0.25 4
('proc_name', 'trange') 0.50 2
('proc_name', 'trange') 0.75 2
('proc_name', 'file_name') 0.10 1
('proc_name', 'file_name') 0.25 1
('proc_name', 'file_name') 0.5

In [39]:
!mkdir -p {log_dir}/bottlenecks

time: 144 ms (started: 2023-03-12 13:58:06 -07:00)


In [40]:
def save_bottlenecks(log_dir: str, bottlenecks: Dict[tuple, object]):
    bottleneck_dir = f"{log_dir}/bottlenecks"
    for view_key, bottleneck_dict in bottlenecks.items():
        file_name = '_'.join(view_key) if isinstance(view_key, tuple) else view_key
        with open(f"{bottleneck_dir}/{file_name}.json", 'w') as json_file:
            json.dump(bottleneck_dict, json_file, cls=NpEncoder, sort_keys=True)

time: 1.44 ms (started: 2023-03-12 13:58:06 -07:00)


In [41]:
save_bottlenecks(log_dir=log_dir, bottlenecks=bottlenecks)

time: 4.14 s (started: 2023-03-12 13:58:07 -07:00)


In [42]:
bottleneck_views[('trange',)]['high_level_view'].compute()

,duration_sum,index_count,size_min,size_max,size_sum,read_count,write_count,metadata_count,read_size,write_size,...,data_size,data_time,open_time,close_time,seek_time,stat_time,duration_pero,duration_perr,duration_score,duration_th
trange,,,,,,,,,,,,,,,,,,,,,
25,0.020422,256,8192,8192,1048576,128,0,128,1048576,0,...,1048576,0.020177,0.001125,0.000000,0,0,0.002532,0.000895,trivial,0.001
59,0.073001,568741,0,8192,1048741,143,0,568598,1048741,0,...,1048741,0.072217,0.000448,0.000248,0,0,0.009051,0.003200,trivial,0.001
60,0.073831,7281,0,11,1243,113,0,7168,1243,0,...,1243,0.073431,0.030508,0.000184,0,0,0.009154,0.003236,trivial,0.001
62,3.381439,74,13405611264,13405611264,992015233536,74,0,0,992015233536,0,...,992015233536,3.381439,0.000000,0.000000,0,0,0.419262,0.148225,medium,0.250
63,3.630890,66,0,13405611264,107244890112,8,0,58,107244890112,0,...,107244890112,3.630890,0.000000,0.000267,0,0,0.450192,0.159160,medium,0.250
64,7.651048,52,0,13405611264,375357115392,28,0,24,375357115392,0,...,375357115392,7.651048,0.000000,0.000179,0,0,0.948649,0.335383,very high,0.750
65,7.981038,18,13405611264,13405611264,241301002752,18,0,0,241301002752,0,...,241301002752,7.981038,0.000000,0.000000,0,0,0.989564,0.349848,very high,0.750
68,0.001185,35,0,0,0,0,0,35,0,0,...,0,0.000000,0.000000,0.001185,0,0,0.000147,0.000052,none,0.000


time: 332 ms (started: 2023-03-12 13:58:12 -07:00)


In [66]:
bottleneck_views[('proc_id',)]['high_level_view'].compute()

,index,duration_sum,index_count,size_min,size_max,size_sum,func_id,read_count,write_count,metadata_count,...,random_size,sequential_time,random_time,data_count,data_size,data_time,duration_pero,duration_perr,duration_score,duration_th
proc_name,,,,,,,,,,,,,,,,,,,,,
app1#localhost#0#35186539112624,26416,2.624664,472,0,13405611264,13405635851,"[['close'], ['closedir', 'readdir'], ['fclose'...",5,0,467,...,0,2.624664,0,5,13405635851,2.622817,0.325430,0.004578,medium,0.25
app1#localhost#1#35186539112624,32354,2.632444,472,0,13405611264,13405635851,"[['close'], ['closedir', 'readdir'], ['fclose'...",5,0,467,...,0,2.632444,0,5,13405635851,2.630800,0.326395,0.004592,medium,0.25
app1#localhost#10#35186539112624,26886,2.542895,472,0,13405611264,13405635851,"[['close'], ['closedir', 'readdir'], ['fclose'...",5,0,467,...,0,2.542895,0,5,13405635851,2.541199,0.315292,0.004436,medium,0.25
app1#localhost#100#35186539112624,31120,2.589109,472,0,13405611264,13405635851,"[['close'], ['closedir', 'readdir'], ['fclose'...",5,0,467,...,0,2.589109,0,5,13405635851,2.587588,0.321022,0.004516,medium,0.25
app1#localhost#101#35186539112624,37089,2.575715,472,0,13405611264,13405635851,"[['close'], ['closedir', 'readdir'], ['fclose'...",5,0,467,...,0,2.575715,0,5,13405635851,2.574267,0.319361,0.004493,medium,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
app1#localhost#95#35186539112624,30933,2.650320,472,0,13405611264,13405635851,"[['close'], ['closedir', 'readdir'], ['fclose'...",5,0,467,...,0,2.650320,0,5,13405635851,2.649052,0.328612,0.004623,medium,0.25
app1#localhost#96#35186539112624,30930,2.761827,472,0,13405611264,13405635851,"[['close'], ['closedir', 'readdir'], ['fclose'...",5,0,467,...,0,2.761827,0,5,13405635851,2.760617,0.342437,0.004817,medium,0.25
app1#localhost#97#35186539112624,30978,2.749732,472,0,13405611264,13405635851,"[['close'], ['closedir', 'readdir'], ['fclose'...",5,0,467,...,0,2.749732,0,5,13405635851,2.748549,0.340938,0.004796,medium,0.25


time: 516 ms (started: 2023-03-12 12:35:33 -07:00)


In [65]:
bottleneck_views[('file_id',)]['high_level_view'].compute()

,index,duration_sum,index_count,size_min,size_max,size_sum,func_id,read_count,write_count,metadata_count,...,random_size,sequential_time,random_time,data_count,data_size,data_time,duration_pero,duration_perr,duration_score,duration_th
file_name,,,,,,,,,,,,,,,,,,,,,
/p/gpfs1/brainusr/datasets/jag/jag100K1vw_train.npy,2586167,8.047040,768,0,13405611264,1715918243200,"[['fclose', 'fopen64'], ['fclose'], ['fread']]",256,0,512,...,0,8.047040,0,256,1715918243200,8.046688,0.997747,0.997338,very high,0.750
/p/gpfs1/haridev/software/lbann/applications/physics/ICF/20210908_125606_jag_wae/experiment.prototext,876352,0.020508,640,8192,8192,3145728,"[['close'], ['open64'], ['read']]",384,0,256,...,0,0.020508,0,384,3145728,0.020252,0.002543,0.002542,trivial,0.001
UNKNOWN,17650845,0.000970,634134,0,0,0,"[['closedir', 'readdir'], ['closedir'], ['read...",0,0,634134,...,0,0.000970,0,0,0,0.000000,0.000120,0.000120,none,0.000


time: 1.38 s (started: 2023-03-12 12:35:25 -07:00)
